In [ ]:
# Load environment variables from .env at the repository root
from dotenv import load_dotenv
load_dotenv()


# Match Documents to Questions

Matching documents to questions using the references for each question.

In [ ]:
from tqdm import tqdm
import pandas as pd
from src.utils.pickle_utils import for_each_pickle_file
from src.utils.document_utils import find_matching_docs
import os

tqdm.pandas()

In [2]:
def add_matching_documents_from_pkl_file(filename, df):
    question_answers_dataset = pd.read_pickle("data/datasets/question_answers_dataset_raw.pkl")
    documents = df["document"].tolist()
    question_answers_dataset["documents"] = question_answers_dataset["references"].apply(
        lambda refs: find_matching_docs(refs, documents, threshold=0.5)
    )
    clean_name = os.path.splitext(os.path.basename(filename))[0]
    question_answers_dataset.to_pickle(f"data/qa_with_docs/question_answers_{clean_name}_dataset.pkl")

for_each_pickle_file("data/preprocessed_documents", add_matching_documents_from_pkl_file)

Processing Pickle files: 100%|██████████| 8/8 [01:49<00:00, 13.65s/it]


## Dataset Validation

### Number of relevant chunks per question

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# dieser globale Container sammelt alle Counts aus jedem einzelnen Pickle
records = []

def validate_qa_with_docs(filename, df):
    # Länge der Liste df["documents"] pro Frage
    doc_counts = df["documents"].apply(len)
    # für jede Count ein Record mit Dateiname
    for cnt in doc_counts:
        records.append({
            "file": filename,
            "doc_count": cnt
        })

# hier wird wie gewohnt über alle Pickles iteriert,
# our callback füllt nur 'records'
for_each_pickle_file("data/qa_with_docs", validate_qa_with_docs)

# nach der Schleife haben wir alles in records – jetzt einmaliges Boxplot
plot_df = pd.DataFrame(records)

sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=plot_df,
    x="file",
    y="doc_count",
    showfliers=False        # Ausreißer ausblenden, optional
)
plt.title("Vergleich: Anzahl Dokumente pro Frage für alle Pickle-Dateien")
plt.xlabel("Pickle-Datei")
plt.ylabel("Anzahl Dokumente pro Frage")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

NameError: name 'for_each_pickle_file' is not defined

### Check Reference to Document Mapping

Checks if the documents mapped to the question contain all the references from that question.

In [4]:
import pandas as pd

def find_missing_reference_matches_in_df(filename, df):
    # Show all columns, rows, and avoid truncating column content
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', None)

    def find_missing_refs(references, documents):
        missing_refs = []

        for ref in references:
            if len(find_matching_docs([ref], documents, threshold=0.5)) == 0:
                missing_refs.append(ref)

        return missing_refs

    # Apply function to extract missing references
    df["missing_refs"] = df.apply(
        lambda row: find_missing_refs(row["references"], row["documents"]), axis=1
    )

    # Only keep rows with missing references
    failed_rows = df[df["missing_refs"].apply(len) > 0]

    print(f"f{filename}: {len(failed_rows)} rows have missing reference matches.")
    print(failed_rows[["question", "references", "missing_refs", "documents"]])

for_each_pickle_file("data/qa_with_docs", find_missing_reference_matches_in_df)

Processing Pickle files:  12%|█▎        | 1/8 [01:16<08:54, 76.33s/it]

fquestion_answers_docs_line_1_0_dataset.pkl: 0 rows have missing reference matches.
Empty DataFrame
Columns: [question, references, missing_refs, documents]
Index: []


Processing Pickle files:  25%|██▌       | 2/8 [02:32<07:38, 76.44s/it]

fquestion_answers_docs_passage_1_0_dataset.pkl: 0 rows have missing reference matches.
Empty DataFrame
Columns: [question, references, missing_refs, documents]
Index: []


Processing Pickle files:  38%|███▊      | 3/8 [03:47<06:19, 75.85s/it]

...

fquestion_answers_docs_word_41_8_dataset.pkl: 0 rows have missing reference matches.
Empty DataFrame
Columns: [question, references, missing_refs, documents]
Index: []


Processing Pickle files:  88%|████████▊ | 7/8 [08:53<01:16, 76.33s/it]

fquestion_answers_docs_word_50_10_dataset.pkl: 0 rows have missing reference matches.
Empty DataFrame
Columns: [question, references, missing_refs, documents]
Index: []


Processing Pickle files: 100%|██████████| 8/8 [10:11<00:00, 76.45s/it]

fquestion_answers_docs_word_800_160_dataset.pkl: 0 rows have missing reference matches.
Empty DataFrame
Columns: [question, references, missing_refs, documents]
Index: []


## Flatten Question Variations

In [5]:
import json
import pandas as pd
from tqdm import tqdm
import os

def flatten(filename, df):
    new_rows = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Flattening Question Variations"):
        # 0) Include the original row itself
        orig = row.copy()
        orig["variant"]       = "default"
        orig["prev_messages"] = []
        new_rows.append(orig)

        # 1) Unpack each variation
        variations = json.loads(row["variations"])
        for variation_key, variation_val in variations.items():
            nr = row.copy()
            nr["variant"] = variation_key

            if variation_key == "contextual" and isinstance(variation_val, list):
                nr["question"]      = variation_val[-1]["message"]
                nr["prev_messages"] = [m["message"] for m in variation_val[:-1]]
            else:
                nr["question"]      = variation_val
                nr["prev_messages"] = []

            new_rows.append(nr)

    # 2) Build your flattened DataFrame and drop 'variations'
    flat_df = (
        pd.DataFrame(new_rows)
        .reset_index(drop=True)
        .drop(columns=["variations"])
    )
    # Strip .pkl if it exists
    clean_name = os.path.splitext(os.path.basename(filename))[0]
    flat_df.to_pickle(f"data/qa_with_docs_flat/{clean_name}_flat.pkl")

for_each_pickle_file("data/qa_with_docs", flatten)

Processing Pickle files: 100%|██████████| 8/8 [00:02<00:00,  3.10it/s]
